# Step 1 - Predict Subject Name

In [1]:
import sys
sys.path.insert(0, '../../../allennlp')
sys.path.insert(0, '../../')
from tqdm import tqdm_notebook
from lib.utils import get_connection 
import random
import numpy as np

tqdm_notebook().pandas()
connection = get_connection()
cursor = connection.cursor()

In [2]:
# Run the end-to-end pipeline on the development set

from lib.simple_qa import load_simple_qa 
from sklearn.utils import shuffle

df_dev, = load_simple_qa(test=True)
df_dev = shuffle(df_dev, random_state=123)
df_dev[:5]

../../lib/simple_qa.py:36: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  filename, header=None, names=['subject', 'relation', 'object', 'question'])


,subject,relation,object,question
17188,02_286,location/place_with_neighborhoods/neighborhoods,075s73,which town is in new york city
4793,01bg1k,baseball/baseball_player/position_s,017drs,does pee wee reese play shortstop or power for...
21187,06tw28,music/artist/track,0vp3fq,What is a track by lutricia mcneal?
18730,0slws_1,music/release_track/recording,0wzyx1,Name a recording by nelson mandela
10014,01m1y,media_common/netflix_genre/titles,0crryw4,Name a film in the netflix genre celtic music.


## Load Subject Name Model

In [3]:
import importlib
import lib.import_notebook
from allennlp.models.archival import load_archive
from allennlp.service.predictors import Predictor
        
ARCHIVE = load_archive('../../pretrained_models/subject_recognition_grid_search_2.02_11_20:56:18/model.tar.gz', cuda_device=0)
PREDICTOR = Predictor.from_archive(ARCHIVE, 'sentence-tagger')

## TEST ##
question = 'what major cities does u.s. route 2 run through ?'
print('Question:', question)
print('Predicted Tags:', PREDICTOR.predict_json({'sentence': question}, 0)['tags'])

/mount/arbeitsdaten46/projekte/dialog-1/kanjurva/Simple-QA-EMNLP-2018/env/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
/mount/arbeitsdaten46/projekte/dialog-1/kanjurva/Simple-QA-EMNLP-2018/env/lib/python3.6/site-packages/allennlp/modules/conditional_random_field.py:37: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  torch.nn.init.xavier_normal(self.transitions)
/mount/arbeitsdaten46/projekte/dialog-1/kanjurva/Simple-QA-EMNLP-2018/env/lib/python3.6/site-packages/allennlp/modules/conditional_random_field.py:38: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(self.start_transitions)
/mount/arbeitsdaten46/projekte/dialog-1/kanjurva/Simple-QA-EMNLP-2018/env/lib/python3.6/site-packages/allennlp/modules/con

Question: what major cities does u.s. route 2 run through ?
Predicted Tags: ['O', 'O', 'O', 'O', 'I', 'I', 'I', 'O', 'O', 'O']


/mount/arbeitsdaten46/projekte/dialog-1/kanjurva/Simple-QA-EMNLP-2018/env/lib/python3.6/site-packages/allennlp/data/fields/text_field.py:119: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  tensor = Variable(torch.LongTensor(padded_array), volatile=not for_training)
/mount/arbeitsdaten46/projekte/dialog-1/kanjurva/Simple-QA-EMNLP-2018/env/lib/python3.6/site-packages/allennlp/modules/encoder_base.py:93: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  num_valid = torch.sum(mask[:, 0]).int().data[0]


## Top K Model Decoder

The best subject name span is not always found in our KG; therefore, here we define a top k viterbi decoder. This allows us to get the top k subject names.

In [4]:
import torch

# FROM: https://gist.github.com/Deepblue129/afaa3613a99a8e7213d2efdd02ae4762 
def viterbi_decode(tag_sequence: torch.Tensor, transition_matrix: torch.Tensor, top_k: int=5):
    """
    Perform Viterbi decoding in log space over a sequence given a transition matrix
    specifying pairwise (transition) potentials between tags and a matrix of shape
    (sequence_length, num_tags) specifying unary potentials for possible tags per
    timestep.
    Parameters
    ----------
    tag_sequence : torch.Tensor, required.
        A tensor of shape (sequence_length, num_tags) representing scores for
        a set of tags over a given sequence.
    transition_matrix : torch.Tensor, required.
        A tensor of shape (num_tags, num_tags) representing the binary potentials
        for transitioning between a given pair of tags.
    top_k : int, required.
        Integer defining the top number of paths to decode.
    Returns
    -------
    viterbi_path : List[int]
        The tag indices of the maximum likelihood tag sequence.
    viterbi_score : float
        The score of the viterbi path.
    """
    sequence_length, num_tags = list(tag_sequence.size())

    path_scores = []
    path_indices = []
    # At the beginning, the maximum number of permutations is 1; therefore, we unsqueeze(0)
    # to allow for 1 permutation.
    path_scores.append(tag_sequence[0, :].unsqueeze(0))
    # assert path_scores[0].size() == (n_permutations, num_tags)

    # Evaluate the scores for all possible paths.
    for timestep in range(1, sequence_length):
        # Add pairwise potentials to current scores.
        # assert path_scores[timestep - 1].size() == (n_permutations, num_tags)
        summed_potentials = path_scores[timestep - 1].unsqueeze(2) + transition_matrix
        summed_potentials = summed_potentials.view(-1, num_tags)

        # Best pairwise potential path score from the previous timestep. 
        max_k = min(summed_potentials.size()[0], top_k)
        scores, paths = torch.topk(summed_potentials, k=max_k, dim=0)
        # assert scores.size() == (n_permutations, num_tags)
        # assert paths.size() == (n_permutations, num_tags)

        scores = tag_sequence[timestep, :] + scores
        # assert scores.size() == (n_permutations, num_tags)
        path_scores.append(scores)
        path_indices.append(paths.squeeze())

    # Construct the most likely sequence backwards.
    path_scores = path_scores[-1].view(-1)
    max_k = min(path_scores.size()[0], top_k)
    viterbi_scores, best_paths = torch.topk(path_scores, k=max_k, dim=0)
    viterbi_paths = []
    for i in range(max_k):
        viterbi_path = [best_paths[i]]
        for backward_timestep in reversed(path_indices):
            viterbi_path.append(int(backward_timestep.view(-1)[viterbi_path[-1]]))
        # Reverse the backward path.
        viterbi_path.reverse()
        # Viterbi paths uses (num_tags * n_permutations) nodes; therefore, we need to modulo.
        viterbi_path = [j % num_tags for j in viterbi_path]
        viterbi_paths.append(viterbi_path)
    return viterbi_paths, viterbi_scores

## TEST ##
sequence_logits = torch.FloatTensor([[1, 0, 0, 4], [1, 0, 6, 2], [0, 3, 0, 4]])
transition_matrix = torch.zeros([4, 4])
transition_matrix[0, 0] = 1
transition_matrix[2, 1] = 5
indices, value = viterbi_decode(sequence_logits, transition_matrix)
assert indices[0] == [3, 2, 1]
assert value[0] == 18

In [5]:
from typing import List

import torch
from torch.autograd import Variable

# Originally From:
# https://github.com/allenai/allennlp/blob/master/allennlp/modules/conditional_random_field.py#L162
def viterbi_tags(logits: List[List[int]], mask: List[int], top_k: int) -> List[List[int]]:
    """
    Uses viterbi algorithm to find most likely tags for the given inputs.
    """
    logits = torch.FloatTensor(logits)
    mask = torch.LongTensor(mask)
    
    max_seq_length, num_tags = logits.size()

    # Augment transitions matrix with start and end transitions
    start_tag = num_tags
    end_tag = num_tags + 1
    transitions = torch.Tensor(num_tags + 2, num_tags + 2).fill_(-10000.)

    transitions[:num_tags, :num_tags] = ARCHIVE.model.crf.transitions.data
    transitions[start_tag, :num_tags] = ARCHIVE.model.crf.start_transitions.data
    transitions[:num_tags, end_tag] = ARCHIVE.model.crf.end_transitions.data

    # Pad the max sequence length by 2 to account for start_tag + end_tag.
    tag_sequence = torch.Tensor(max_seq_length + 2, num_tags + 2)

    sequence_length = torch.sum(mask)

    # Start with everything totally unlikely
    tag_sequence.fill_(-10000.)
    # At timestep 0 we must have the START_TAG
    tag_sequence[0, start_tag] = 0.
    # At steps 1, ..., sequence_length we just use the incoming logits
    logits[:sequence_length]
    tag_sequence[1:(sequence_length + 1), :num_tags] = logits[:sequence_length]
    # And at the last timestep we must have the END_TAG
    tag_sequence[sequence_length + 1, end_tag] = 0.

    # We pass the tags and the transitions to ``viterbi_decode``.
    viterbi_paths, viterbi_scores = viterbi_decode(tag_sequence[:(sequence_length + 2)], transitions, top_k)
    # Get rid of START and END sentinels and append.
    viterbi_paths = [path[1:-1] for path in viterbi_paths]
    # Ensure that hidden tokens START and END are not in path
    viterbi_paths = [path for path in viterbi_paths if 2 not in path and 3 not in path]
    # Translate indexes to labels
    viterbi_paths = [
        [ARCHIVE.model.vocab.get_token_from_index(i, namespace="labels")
         for i in paths] for paths in viterbi_paths
    ]
    return viterbi_paths, viterbi_scores

## TEST ##
top_k = 5
predicted = PREDICTOR.predict_json({'sentence': 'what major cities does u.s. route 2 run through ?'}, 0)
viterbi_paths, viterbi_scores = viterbi_tags(predicted['logits'], predicted['mask'], top_k)

# Best viterbi_paths should equal to predicted tags
assert predicted['tags'] == viterbi_paths[0]

for i in range(top_k):
    print('[Score: %f] Path:' % viterbi_scores[i], viterbi_paths[i])

[Score: 101.382729] Path: ['O', 'O', 'O', 'O', 'I', 'I', 'I', 'O', 'O', 'O']
[Score: 83.410515] Path: ['O', 'O', 'O', 'O', 'I', 'I', 'I', 'O', 'I', 'O']
[Score: 82.760170] Path: ['O', 'O', 'O', 'O', 'I', 'I', 'O', 'O', 'O', 'O']
[Score: 82.228661] Path: ['O', 'O', 'O', 'O', 'I', 'O', 'I', 'O', 'O', 'O']
[Score: 81.733078] Path: ['O', 'O', 'O', 'O', 'I', 'I', 'I', 'O', 'O', 'I']


## Wrap Up

Put the model and top k decoder together. Predict the subject name accross all the examples in the dataframe.

In [6]:
import math
import re

def predict_subject_name(tokens, top_k=500):
    # Predict Tags
    predicted = PREDICTOR.predict_json({'sentence': ' '.join(tokens)}, 0)
    viterbi_paths, viterbi_scores = viterbi_tags(predicted['logits'], predicted['mask'], top_k)
    
    predicted_subject_names = []
    for tags, score in zip(viterbi_paths, viterbi_scores):
        assert len(tags) == len(tokens)
        # Ignore if multiple subject names are selected
        n_subjects = sum(tags[i] == 'I' and (i - 1 == -1 or tags[i - 1] == 'O') for i in range(len(tags)))
        if n_subjects == 1:
            predicted_subject_name = ' '.join([tokens[i] for i, tag in
                                               enumerate(tags) if tag == 'I'])
            start_index = [i for i, tag in enumerate(tags) 
                           if tag == 'I' and (i == 0 or tags[i - 1] == 'O')][0]
            end_index = [i for i, tag in enumerate(tags) 
                         if tag == 'I' and (i == len(tags) - 1 or tags[i + 1] == 'O')][0] + 1
            predicted_subject_names.append({
                'name': predicted_subject_name,
                'score': score,
                'start_index': start_index,
                'end_index': end_index,
            })
    return predicted_subject_names

## TEST ##
print('Sample Output:')
predict_subject_name(['what', 'major', 'cities', 'does', 'u.s.', 'route', '2', 'run', 'through', '?'])

Sample Output:


[{'name': 'u.s. route 2',
  'score': tensor(101.3827),
  'start_index': 4,
  'end_index': 7},
 {'name': 'u.s. route',
  'score': tensor(82.7602),
  'start_index': 4,
  'end_index': 6},
 {'name': 'does u.s. route 2',
  'score': tensor(80.7309),
  'start_index': 3,
  'end_index': 7},
 {'name': 'route 2',
  'score': tensor(80.7288),
  'start_index': 5,
  'end_index': 7},
 {'name': 'u.s. route 2 run',
  'score': tensor(79.8302),
  'start_index': 4,
  'end_index': 8},
 {'name': 'u.s.', 'score': tensor(65.8604), 'start_index': 4, 'end_index': 5},
 {'name': 'u.s. route 2 run through',
  'score': tensor(64.1123),
  'start_index': 4,
  'end_index': 9},
 {'name': '2', 'score': tensor(63.8291), 'start_index': 6, 'end_index': 7},
 {'name': 'does u.s. route',
  'score': tensor(62.1083),
  'start_index': 3,
  'end_index': 6},
 {'name': 'route', 'score': tensor(62.1063), 'start_index': 5, 'end_index': 6},
 {'name': 'cities does u.s. route 2',
  'score': tensor(60.5656),
  'start_index': 2,
  'end_ind

In [7]:
import pandas as pd
from numpy import nan


PREPROCESS = importlib.import_module(
                "notebooks.Simple QA Models.Subject Recognition Data").preprocess
TOKENIZE = importlib.import_module(
                "notebooks.Simple QA Models.Subject Recognition Data").spacy_tokenize

def add_predicted_subject_name(row):
    question_tokens = TOKENIZE(PREPROCESS(row['question']))
    predicted_subject_names = predict_subject_name(question_tokens)
    row['predicted_subject_names'] = predicted_subject_names
    row['predicted_question_tokens'] = question_tokens
    return row

df_dev = df_dev.progress_apply(add_predicted_subject_name, axis=1)
df_dev[:5]

importing Jupyter notebook from ../../notebooks/Simple QA Models/Subject Recognition Data.ipynb


importing Jupyter notebook from ../../notebooks/Simple QA Numbers/HYPOTHESIS - Subject Name not in Question.ipynb



AttributeError: 'LazyConfigValue' object has no attribute 'lower'

       subject                                         relation   object  \
17188   02_286  location/place_with_neighborhoods/neighborhoods   075s73   
4793    01bg1k              baseball/baseball_player/position_s   017drs   
21187   06tw28                               music/artist/track   0vp3fq   
18730  0slws_1                    music/release_track/recording   0wzyx1   
10014    01m1y                media_common/netflix_genre/titles  0crryw4   

                                                question  \
17188                     which town is in new york city   
4793   does pee wee reese play shortstop or power for...   
21187                What is a track by lutricia mcneal?   
18730                 Name a recording by nelson mandela   
10014     Name a film in the netflix genre celtic music.   

                                 predicted_subject_names  \
17188  [{'name': 'new york city', 'score': tensor(64....   
4793   [{'name': 'pee wee reese', 'score': tensor(91....   
21

## Analysis Setup

Add the True `subject_name` and `start_index` / `end_index` to check the accuracy of our predicted values.

In [8]:
import importlib
from functools import partial

edit_distance_link_alias = importlib.import_module(
                "notebooks.Simple QA Numbers.HYPOTHESIS - Question Refers to Multiple Subjects").edit_distance_link_alias
normalize = importlib.import_module(
                "notebooks.Simple QA Numbers.HYPOTHESIS - Subject Name not in Question").normalize

# Create a column with the subject_name linked per example
df_dev['subject_name'] = df_dev.progress_apply(partial(edit_distance_link_alias, cursor, normalize), axis=1)

importing Jupyter notebook from ../../notebooks/Simple QA Numbers/HYPOTHESIS - Question Refers to Multiple Subjects.ipynb


In [9]:
import importlib
# Adds:
# - question_tokens
# - subject_name
# - start_index
# - end_index
find_subject_name_span = importlib.import_module(
                "notebooks.Simple QA Models.Subject Recognition Data").find_subject_name_span

# Create a column with the subject_name linked per example
df_dev = df_dev.progress_apply(find_subject_name_span, axis=1)
df_dev[:5]

AttributeError: 'LazyConfigValue' object has no attribute 'lower'

       end_index   object                          predicted_question_tokens  \
17188        7.0   075s73             [which, town, is, in, new, york, city]   
4793         4.0   017drs  [does, pee, wee, reese, play, shortstop, or, p...   
21187        7.0   0vp3fq      [what, is, a, track, by, lutricia, mcneal, ?]   
18730        6.0   0wzyx1          [name, a, recording, by, nelson, mandela]   
10014        9.0  0crryw4  [name, a, film, in, the, netflix, genre, celti...   

                                 predicted_subject_names  \
17188  [{'name': 'new york city', 'score': tensor(64....   
4793   [{'name': 'pee wee reese', 'score': tensor(91....   
21187  [{'name': 'lutricia mcneal', 'score': tensor(7...   
18730  [{'name': 'nelson mandela', 'score': tensor(57...   
10014  [{'name': 'celtic music', 'score': tensor(106....   

                                                question  \
17188                     which town is in new york city   
4793   does pee wee reese play shortst

In [10]:
df_dev.to_pickle('step_1_predict_subject_name.pkl')

In [11]:
import pandas as pd

df_dev = pd.read_pickle('step_1_predict_subject_name.pkl')

## Analysis - Correct Span

We determine the correct subject name span and compare it to the predicted subject name.

In [12]:
from tqdm import tqdm_notebook

accuracies = [0] * 100
total = 0

def is_correct(row, top_k):
    for i in range(min(top_k, len(row['predicted_subject_names']))):
        if (row['start_index'] == row['predicted_subject_names'][i]['start_index'] and
            row['end_index'] == row['predicted_subject_names'][i]['end_index']):
            return True
    return False
    

for index, row in tqdm_notebook(df_dev.iterrows(), total=df_dev.shape[0]):
    if not isinstance(row['subject_name'], str):
        continue
        
    total += 1
    accuracies = [count + is_correct(row, i + 1) for i, count in enumerate(accuracies)]

for i, count in enumerate(accuracies):
    print('Accuracy Top %d: %f [%d of %d]' % (i + 1, count / total, count, total))


Accuracy Top 1: 0.954999 [20309 of 21266]
Accuracy Top 2: 0.973432 [20701 of 21266]
Accuracy Top 3: 0.980626 [20854 of 21266]
Accuracy Top 4: 0.984811 [20943 of 21266]
Accuracy Top 5: 0.987539 [21001 of 21266]
Accuracy Top 6: 0.989091 [21034 of 21266]
Accuracy Top 7: 0.991019 [21075 of 21266]
Accuracy Top 8: 0.991959 [21095 of 21266]
Accuracy Top 9: 0.992899 [21115 of 21266]
Accuracy Top 10: 0.993746 [21133 of 21266]
Accuracy Top 11: 0.994498 [21149 of 21266]
Accuracy Top 12: 0.994921 [21158 of 21266]
Accuracy Top 13: 0.995251 [21165 of 21266]
Accuracy Top 14: 0.995627 [21173 of 21266]
Accuracy Top 15: 0.995815 [21177 of 21266]
Accuracy Top 16: 0.996144 [21184 of 21266]
Accuracy Top 17: 0.996238 [21186 of 21266]
Accuracy Top 18: 0.996755 [21197 of 21266]
Accuracy Top 19: 0.997414 [21211 of 21266]
Accuracy Top 20: 0.997555 [21214 of 21266]
Accuracy Top 21: 0.997884 [21221 of 21266]
Accuracy Top 22: 0.997978 [21223 of 21266]
Accuracy Top 23: 0.998072 [21225 of 21266]
Accuracy Top 24: 0.

## Analysis - Normalized Link

We normalize the correct subject name and compare it to the predicted name normalized.

We expect this to be lower than the span analysis because in "HYPOTHESIS - Subject Name not in Question", we found that 97.85% of the time the subject name normalized is not in the question; therefore, some spans that are correct will not be equal to the subject name normalized.

In [13]:
import importlib

normalize = importlib.import_module(
                "notebooks.Simple QA Numbers.HYPOTHESIS - Subject Name not in Question").normalize
tokenize = importlib.import_module(
                "notebooks.Simple QA Models.Subject Recognition Data").spacy_tokenize

In [14]:
from tqdm import tqdm_notebook
from lib.utils import format_pipe_table

negative_sample = []
accuracies = [0] * 10
total = 0

def is_correct(row, top_k):
    subject_name = normalize(' '.join(tokenize(row['subject_name'])))
    for i in range(min(top_k, len(row['predicted_subject_names']))):
        predicted_subject_name = normalize(row['predicted_subject_names'][i]['name'])
        if predicted_subject_name == subject_name:
            return True
    return False

for index, row in tqdm_notebook(df_dev.iterrows(), total=df_dev.shape[0]):
    if not isinstance(row['subject_name'], str):
        continue
        
    total += 1
    accuracies = [count + is_correct(row, i + 1) for i, count in enumerate(accuracies)]
    if not is_correct(row, 1):
        negative_sample.append({
            'Subject Name': row['subject_name'],
            'Predicted Subject Name': [row['predicted_subject_names'][i]['name']
                                       for i in range(min(5, len(row['predicted_subject_names'])))],
        })

for i, count in enumerate(accuracies):
    print('Accuracy Top %d: %f [%d of %d]' % (i + 1, count / total, count, total))
print('Negative Sample:\n')
print(format_pipe_table(negative_sample[:50]))


Accuracy Top 1: 0.943102 [20056 of 21266]
Accuracy Top 2: 0.959795 [20411 of 21266]
Accuracy Top 3: 0.966754 [20559 of 21266]
Accuracy Top 4: 0.970940 [20648 of 21266]
Accuracy Top 5: 0.973479 [20702 of 21266]
Accuracy Top 6: 0.975031 [20735 of 21266]
Accuracy Top 7: 0.976582 [20768 of 21266]
Accuracy Top 8: 0.977570 [20789 of 21266]
Accuracy Top 9: 0.978604 [20811 of 21266]
Accuracy Top 10: 0.979357 [20827 of 21266]
Negative Sample:

| Index | Predicted Subject Name | Subject Name |
| --- | --- | --- |
| 0 | ['enforcement', 'enforcement agency', 'law enforcement', 'law enforcement agency', 'agency'] | law enforcement agency |
| 1 | ['outlying islands', 'states minor outlying islands', 'islands', 'united states minor outlying islands', 'minor outlying islands'] | united states minor outlying islands |
| 2 | ['the greatest hits', 'the greatest hits album', 'greatest hits', 'greatest hits album', 'the greatest'] | greatest hits |
| 3 | ['heroines mantle', 'heroines', 'mantle', 'heroines